<a href="https://colab.research.google.com/github/druzxh/flask-ipynb/blob/main/flask_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Flask

In [ ]:
!pip install Flask Flask-Cors reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.5 MB/s eta 0:00:00


# Save Token

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2iUWiPXkNqJu5H4iaaJ1h9ldOcc_5aRvaJ8LvrLsutQCeZZ2t

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask-swagger-ui
# from google.colab import files
# uploaded = files.upload()

# !mkdir -p static
# !mv swagger.json static/


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.5 MB/s eta 0:00:00


# Import Flask, Ngrok

In [ ]:
import os
from flask import Flask, jsonify, redirect, request
from flask_cors import CORS
from pyngrok import ngrok
from flask_swagger_ui import get_swaggerui_blueprint
import json

app = Flask(__name__)
CORS(app)

SWAGGER_URL = '/api/docs'
API_URL = '/static/swagger.json'

class SwaggerGenerator:
    def __init__(self, app, swagger_url, api_url, app_name):
        self.app = app
        self.swagger_url = swagger_url
        self.api_url = api_url
        self.app_name = app_name
        self.swagger_spec = {
            "swagger": "2.0",
            "info": {
                "description": f"API {self.app_name}",
                "version": "1.0.0",
                "title": self.app_name
            },
            "host": "",
            "basePath": "/",
            "tags": [
                {"name": "test", "description": "Operations related to test"},
                {"name": "users", "description": "Operations related to users"},
                {"name": "categories", "description": "Operations related to categories"}
            ],
            "paths": {}
        }

    def register_swagger_ui(self):
        swaggerui_blueprint = get_swaggerui_blueprint(
            self.swagger_url,
            self.api_url,
            config={'app_name': self.app_name}
        )
        self.app.register_blueprint(swaggerui_blueprint, url_prefix=self.swagger_url)

    def register_path(self, route, endpoint, methods=['GET'], tags=None):
        if tags is None:
            tags = []

        route_data = {
            "tags": tags,
            "summary": f"Returns {endpoint} page",
            "description": f"Returns {endpoint} page",
            "operationId": endpoint,
            "produces": ["application/json"],
            "responses": {
                "200": {
                    "description": "successful operation",
                    "schema": {"$ref": f"#/definitions/{endpoint.capitalize()[:-1]}"}
                }
            }
        }
        if route not in self.swagger_spec['paths']:
            self.swagger_spec['paths'][route] = {}
        for method in methods:
            self.swagger_spec['paths'][route][method.lower()] = route_data

    def set_host(self, host):
        self.swagger_spec['host'] = host

    def generate_swagger(self):
        with open(os.path.join(app.root_path, 'static', 'swagger.json'), 'w') as f:
            json.dump(self.swagger_spec, f, indent=4)

        self.register_swagger_ui()

@app.route('/')
def index():
    return redirect(SWAGGER_URL)

@app.route('/api/test', methods=['GET', 'POST'])  # Example of multiple methods
def test():
    if request.method == 'GET':
        data = {
            'message': 'Hello, World!',
            'status': 'success'
        }
        return jsonify(data)
    elif request.method == 'POST':
        data = request.get_json()
        return jsonify(data)

@app.route('/api/users', methods=['GET', 'POST'])
def users():
    if request.method == 'GET':
        users = [
            {"id": 1, "username": "user1"},
            {"id": 2, "username": "user2"}
        ]
        return jsonify(users)
    elif request.method == 'POST':
        data = request.get_json()
        return jsonify(data), 201

@app.route('/api/categories', methods=['GET', 'POST'])
def categories():
    if request.method == 'GET':
        categories = [
            {"id": 1, "name": "category1"},
            {"id": 2, "name": "category2"}
        ]
        return jsonify(categories)
    elif request.method == 'POST':
        data = request.get_json()
        return jsonify(data), 201

if __name__ == '__main__':
    os.makedirs(os.path.join(app.root_path, 'static'), exist_ok=True)

    swagger_generator = SwaggerGenerator(app, SWAGGER_URL, API_URL, "API CLEANSING DATA")
    swagger_generator.register_path('/api/test', 'test', methods=['GET', 'POST'], tags=['test'])
    swagger_generator.register_path('/api/users', 'users', methods=['GET', 'POST'], tags=['users'])
    swagger_generator.register_path('/api/users/create', 'users', methods=['POST'], tags=['users'])
    swagger_generator.register_path('/api/categories', 'categories', methods=['GET', 'POST'], tags=['categories'])
    swagger_generator.register_path('/api/categories/create', 'categories', methods=['POST'], tags=['categories'])

    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f'Public URL: {public_url}{SWAGGER_URL}')

    swagger_generator.set_host(public_url.replace('http://', '').replace('https://', ''))
    swagger_generator.generate_swagger()

    app.run(port=port)


Public URL: https://f9d4-34-125-115-42.ngrok-free.app/api/docs
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:20] "GET /api/docs HTTP/1.1" 308 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:21] "GET /api/docs/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:22] "GET /api/docs/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:22] "GET /api/docs/swagger-ui.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:22] "GET /api/docs/swagger-ui-bundle.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:22] "GET /api/docs/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:25] "GET /api/docs/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jun/2024 07:53:26] "GET /api/docs/swagger-ui-bundle.js HTTP/1.1" 200 -
INFO:wer